In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from time import time
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.neural_network import MLPRegressor


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  

In [7]:
train = pd.read_csv('train_modified.csv')
test = pd.read_csv('test_modified.csv')


df_train_target = train['SalePrice'].values
df_train_data = train.drop(['SalePrice'],axis = 1).values


df_train_target = train['SalePrice'].values
df_train_data = train.drop(['SalePrice'],axis = 1).values
print 'df_train_data shape is ', df_train_data.shape
print 'df_train_target shape is ', df_train_target.shape

target='SalePrice'
IDcol = 'Id'

# 一、Fix learning rate and number of estimators for tuning tree-based parameters

In order to decide on boosting parameters, we need to set some initial values of other parameters. Lets take the following values:

max_depth = 5 : This should be between 3-10. I’ve started with 5 but you can choose a different number as well. 4-6 can be good starting points.
min_child_weight = 1 : A smaller value is chosen because it is a highly imbalanced class problem and leaf nodes can have smaller size groups.
gamma = 0 : A smaller value like 0.1-0.2 can also be chosen for starting. This will anyways be tuned later.
subsample, colsample_bytree = 0.8 : This is a commonly used used start value. Typical values range between 0.5-0.9.
scale_pos_weight = 1: Because of high class imbalance.
Please note that all the above are just initial estimates and will be tuned later. Lets take the default learning rate of 0.1 here and check the optimum number of trees using cv function of xgboost. The function defined above will do it for us.

In [5]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,metrics='auc')
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['SalePrice'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['SalePrice'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['SalePrice'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBRegressor(max_depth=3, 
                    learning_rate=0.1, 
                    n_estimators=100, 
                    silent=True, 
                    objective='reg:linear', 
                    nthread=-1, gamma=0, 
                    min_child_weight=1, 
                    max_delta_step=0, 
                    subsample=1, 
                    colsample_bytree=1, 
#                    colsample_bylevel=1, 
#                    reg_alpha=0, 
#                    reg_lambda=1, 
#                    scale_pos_weight=1, 
                    base_score=0.5, 
                    seed=0, 
                    missing=None)



modelfit(xgb1, train, predictors)

In [16]:
xgb1 = XGBRegressor(max_depth=3, 
                    learning_rate=0.1, 
                    n_estimators=100, 
                    silent=True, 
                    objective='reg:linear', 
                    nthread=-1, gamma=0, 
                    min_child_weight=1, 
                    max_delta_step=0, 
                    subsample=1, 
                    colsample_bytree=1, 
#                    colsample_bylevel=1, 
#                    reg_alpha=0, 
#                    reg_lambda=1, 
#                    scale_pos_weight=1, 
                    base_score=0.5, 
                    seed=0, 
                    missing=None).fit(df_train_data, df_train_target)


cc = xgb1.predict(df_train_data)

fpr, tpr, thresholds = metrics.roc_curve(df_train_target, cc,pos_label=)
metrics.auc(fpr, tpr)

ValueError: Data is not binary and pos_label is not specified

In [17]:
? metrics.roc_curve